# 1. Obtain Pokemon images

## 1.1 Obtain the images from Kaggle

In [ ]:
!kaggle datasets download -d thedagger/pokemon-generation-one -p data/images

In [ ]:
!unzip data/images/pokemon-generation-one.zip -d data/images

## 1.2 Show some images with its label

In [ ]:
import os
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image_folder = 'data/images/dataset'

pokemon_folders = [f for f in os.listdir(image_folder)]

fig, ax = plt.subplots(2, 4, figsize=(8, 8))
ax = ax.flatten()

for i in range(8):
    random_pokemon = random.choice(pokemon_folders)
    pokemon_images = os.listdir(os.path.join(image_folder, random_pokemon))
    random_image = random.choice(pokemon_images)
    image_path = os.path.join(image_folder, random_pokemon, random_image)
    
    img = mpimg.imread(image_path)
    ax[i].imshow(img)
    ax[i].set_title(random_pokemon)
    ax[i].axis('off') 

plt.tight_layout()
plt.show()


## 1.3 See the class distribution

In [ ]:
image_counts = {}
for folder in pokemon_folders:
    folder_path = os.path.join(image_folder, folder) # Obtiene la ruta 
    image_files = [f for f in os.listdir(folder_path)]
    image_counts[folder] = len(image_files)

image_counts = dict(sorted(image_counts.items(), key=lambda item: item[1], reverse=True))

top_20_pokemon = list(image_counts.keys())[:20]
top_20_counts = list(image_counts.values())[:20]

bottom_20_pokemon = list(image_counts.keys())[-20:]
bottom_20_counts = list(image_counts.values())[-20:]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4)) 

ax1.bar(top_20_pokemon, top_20_counts, color='green')
ax1.set_title('Top 20 Pokémon with Most Images')
ax1.set_xlabel('Pokémon')
ax1.set_ylabel('Number of Images')
ax1.tick_params(axis='x', rotation=90) 

ax2.bar(bottom_20_pokemon, bottom_20_counts, color='red')
ax2.set_title('Top 20 Pokémon with Fewest Images')
ax2.set_xlabel('Pokémon')
ax2.set_ylabel('Number of Images')
ax2.tick_params(axis='x', rotation=90)

plt.tight_layout()

## 1.4 Divide into train, validation and test

In [ ]:
import os
import shutil
import random
from pathlib import Path
from PIL import Image

original_dir = Path(image_folder)
new_base_dir = original_dir.parent / 'splitted_dataset'

def remove_iccp_if_png(image_path):
    if image_path.suffix.lower() == '.png':
        with Image.open(image_path) as img:
            img.save(image_path, 'PNG', icc_profile=None)

def copy_and_process_images(images, src_folder, dest_folder, pokemon):
    os.makedirs(dest_folder / pokemon, exist_ok=True)
    for img in images:
        src_img_path = src_folder / img
        dest_img_path = dest_folder / pokemon / img
        shutil.copyfile(src_img_path, dest_img_path)
        remove_iccp_if_png(dest_img_path)

def split_dataset(train_ratio=0.7, val_ratio=0.2):
    for split in ['train', 'val', 'test']:
        os.makedirs(new_base_dir / split, exist_ok=True)

    for pokemon in os.listdir(original_dir):
        pokemon_path = original_dir / pokemon
        images = os.listdir(pokemon_path)
        random.shuffle(images)

        total_images = len(images)
        train_count = int(total_images * train_ratio)
        val_count = int(total_images * val_ratio)

        train_images = images[:train_count]
        val_images = images[train_count:train_count + val_count]
        test_images = images[train_count + val_count:]

        copy_and_process_images(train_images, pokemon_path, new_base_dir / 'train', pokemon)
        copy_and_process_images(val_images, pokemon_path, new_base_dir / 'val', pokemon)
        copy_and_process_images(test_images, pokemon_path, new_base_dir / 'test', pokemon)

split_dataset()


## 1.5 Load dataset from directory

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from pathlib import Path

image_folder = 'data/images/dataset'
original_dir = Path(image_folder)
new_base_dir = original_dir.parent / 'splitted_dataset'

train_dir = new_base_dir / 'train'
val_dir = new_base_dir / 'val'
test_dir = new_base_dir / 'test'

train_dataset = image_dataset_from_directory(train_dir,
                batch_size=32,
                image_size=(180, 180))

val_dataset = image_dataset_from_directory(val_dir,
                batch_size=32,
                image_size=(180, 180))

test_dataset = image_dataset_from_directory(test_dir,
                batch_size=32,
                image_size=(180, 180))

# 2. Model and training

## 2.1 Basic model definition

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(149, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

## 2.2 Model compilation

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
            optimizer="rmsprop",
            metrics=["accuracy"])

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint(
    "models/basic_pokemon_model.keras",
    save_best_only=True,
    monitor="val_loss"
)]

## 2.3 Model training

In [ ]:
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks
)

## 2.4 Model evaluation

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.figure()

plt.show()

## 2.5 Test the model

In [ ]:
test_model = keras.models.load_model("models/basic_pokemon_model.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")

# 3. Regularizate the basic model

## 3.1 Use data augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.2),
        layers.RandomTranslation(0.1, 0.1)
    ]
)

In [ ]:
plt.figure(figsize=(10,10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(149, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="sparse_categorical_crossentropy",
            optimizer="rmsprop",
            metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(
    "models/weight_regularized_pokemon_model.keras",
    save_best_only=True,
    monitor="val_loss"
)]


history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.figure()

plt.show()

In [ ]:
test_model = keras.models.load_model("models/weight_regularized_pokemon_model.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")

## 3.2 Feature extraction

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)
conv_base.trainable = False

conv_base.summary()

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
  
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)                      
x = keras.applications.vgg16.preprocess_input(x)   
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(149, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_pokemon_model.keras",
        save_best_only=True,
        monitor="val_loss"),
    
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True)  
]

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.figure()

plt.show()

In [ ]:
test_model = keras.models.load_model("feature_extraction_pokemon_model.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")

## 3.3 Fine Tuning

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, optimizers

conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)

conv_base.trainable = True

for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomTranslation(0.1, 0.1),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)                      
x = keras.applications.vgg16.preprocess_input(x)   
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(149, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizers.Adam(learning_rate=1e-5),
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning_pokemon.keras",
        save_best_only=True,
        monitor="val_loss"),
    
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True)  
]

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.figure()

plt.show()

In [ ]:
test_model = keras.models.load_model("fine_tuning_pokemon.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc}")